In [1]:
"""ОБРОБКА УСІХ ЗОБРАЖЕНЬ ДАТАСЕТУ ВЕЙВЛЕТНИМИ ПЕРЕТВОРЕННЯМИ"""

import cv2
import numpy as np
import pywt
import os
from tqdm import tqdm

def apply_wavelet_centered(img, wavelet='db6', lambda_=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coeffs2 = pywt.dwt2(gray, wavelet)
    _, (H, V, D) = coeffs2
    W = np.sqrt(H ** 2 + V ** 2 + D ** 2)
    W = cv2.resize(W, (img.shape[1], img.shape[0]))
    W_centered = W - np.mean(W)
    img_float = img.astype(np.float32)
    W3 = np.stack([W_centered] * 3, axis=-1)
    result = img_float + lambda_ * W3
    return np.clip(result, 0, 255).astype(np.uint8)

def preprocess_folder(input_folder, output_folder, lambda_=0.5):
    os.makedirs(output_folder, exist_ok=True)
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith(('.jpg', '.png')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue
            img_wavelet = apply_wavelet_centered(img, lambda_=lambda_)
            cv2.imwrite(os.path.join(output_folder, filename), img_wavelet)

def process_all_sets(base_input, base_output, lambda_=0.5):
    for split in ['train', 'valid', 'test']:
        preprocess_folder(
            os.path.join(base_input, split, 'images'),
            os.path.join(base_output, split, 'images'),
            lambda_
        )
        labels_src = os.path.join(base_input, split, 'labels')
        labels_dst = os.path.join(base_output, split, 'labels')
        os.makedirs(labels_dst, exist_ok=True)
        for f in os.listdir(labels_src):
            src = os.path.join(labels_src, f)
            dst = os.path.join(labels_dst, f)
            if not os.path.exists(dst):
                with open(src, 'r') as r, open(dst, 'w') as w:
                    w.write(r.read())

if __name__ == '__main__':
    original = 'orig_data'
    processed = 'data'
    process_all_sets(original, processed, lambda_=0.5)


100%|██████████| 13/13 [00:00<00:00, 22.45it/s]
